<a href="https://colab.research.google.com/github/blindauth/abstention_experiments/blob/master/diabetic_retinopathy/AnalyzeDiabeticRetinopathyResults.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!rm -r abstention
!pip uninstall abstention
!git clone https://github.com/blindauth/abstention.git
!pip install abstention/
![[ -e abstention_experiments ]] || git clone https://github.com/blindauth/abstention_experiments.git

rm: cannot remove 'abstention': No such file or directory
Cloning into 'abstention'...
remote: Enumerating objects: 51, done.
remote: Counting objects: 100% (51/51), done.
remote: Compressing objects: 100% (38/38), done.
remote: Total 51 (delta 18), reused 42 (delta 9), pack-reused 0
Unpacking objects: 100% (51/51), done.
Processing ./abstention
  Stored in directory: /tmp/pip-ephem-wheel-cache-0rqzb2k2/wheels/60/8e/d2/9e9ca02e7b5f76bfda2e2daa6dcbe42c19095c502ccb653729
Successfully built abstention
Cloning into 'abstention_experiments'...
remote: Enumerating objects: 1084, done.
remote: Counting objects: 100% (1084/1084), done.
remote: Compressing objects: 100% (1079/1079), done.
remote: Total 4022 (delta 25), reused 1050 (delta 5), pack-reused 2938
Receiving objects: 100% (4022/4022), 289.18 MiB | 46.25 MiB/s, done.
Resolving deltas: 100% (41/41), done.
Checking out files: 100% (3891/3891), done.


In [2]:
!ls abstention_experiments/diabetic_retinopathy

AnalyzeDiabeticRetinopathyResults.ipynb
diabeticretinopathy_useweightrescalpreds-False_labelshift-False_abstention_results.json.gz
diabeticretinopathy_useweightrescalpreds-False_labelshift-True_abstention_results.json.gz
diabeticretinopathy_useweightrescalpreds-True_labelshift-False_abstention_results.json.gz
diabeticretinopathy_useweightrescalpreds-True_labelshift-True_abstention_results.json.gz
flip-False_rotamt-0
flip-False_rotamt-180
flip-False_rotamt-270
flip-False_rotamt-90
flip-True_rotamt-0
flip-True_rotamt-180
flip-True_rotamt-270
flip-True_rotamt-90
RunDiabeticRetinopathyExperiments.ipynb
valid_labels.txt.gz


In [0]:
import gzip
import json

In [0]:
dr_weightrescale_false_labelshift_false_abstention_results =\
  json.loads(gzip.open("abstention_experiments/diabetic_retinopathy/"
    +"diabeticretinopathy_useweightrescalpreds"
    +"-False_labelshift-False_abstention_results.json.gz").read())
dr_weightrescale_false_labelshift_true_abstention_results =\
  json.loads(gzip.open("abstention_experiments/diabetic_retinopathy/"
    +"diabeticretinopathy_useweightrescalpreds"
    +"-False_labelshift-True_abstention_results.json.gz").read())

dr_weightrescale_true_labelshift_false_abstention_results =\
  json.loads(gzip.open("abstention_experiments/diabetic_retinopathy/"
    +"diabeticretinopathy_useweightrescalpreds"
    +"-True_labelshift-False_abstention_results.json.gz").read())
dr_weightrescale_true_labelshift_true_abstention_results =\
  json.loads(gzip.open("abstention_experiments/diabetic_retinopathy/"
    +"diabeticretinopathy_useweightrescalpreds"
    +"-True_labelshift-True_abstention_results.json.gz").read())

In [32]:
from abstention.figure_making_utils import (
    wilcox_srs, get_ustats_mat,
    get_tied_top_and_worst_methods)
import numpy as np

abstfracs = ['0.3', '0.25', '0.2', '0.15', '0.1']

methodnames_noadapt = [
    'yescalib:estmarginalweightedkappa',
    'yescalib:jsdiv',
    'yescalib:entropy',
    'nocalib:entropy',
    'yescalib:distmaxclassprobfromone',
    'nocalib:distmaxclassprobfromone',
    'yescalib:variance',
    'nocalib:variance'    
]

methodnames_yesadapt = [
    'yesadapt:estmarginalweightedkappa',
    'noadapt:estmarginalweightedkappa',
    'yesadapt:jsdiv',
    'noadapt:jsdiv',
    'yesadapt:entropy',
    'noadapt:entropy',
    'yesadapt:distmaxclassprobfromone',
    'noadapt:distmaxclassprobfromone',
    'yesadapt:variance',
    'noadapt:variance'    
]

abstname_to_nicename = {
    'estmarginalweightedkappa': 'Est $\Delta$Metric (Ours)',
    'jsdiv': 'JS Div. (Ours)',
    'entropy': 'Entropy',
    'distmaxclassprobfromone': 'Max Class Prob.',
    'variance': 'Test-time Dropout'}

def stdderr(vals):
  return np.std(vals, ddof=1)/np.sqrt(len(vals))

for settingname, results_dict, methodnames in [
    ("yesweightrescale_nolabelshift",
     dr_weightrescale_true_labelshift_false_abstention_results,
     methodnames_noadapt),
    ("yesweightrescale_yeslabelshift",
     dr_weightrescale_true_labelshift_true_abstention_results,
     methodnames_yesadapt),
    ("noweightrescale_nolabelshift",
     dr_weightrescale_false_labelshift_false_abstention_results,
     methodnames_noadapt),
    ("noweightrescale_yeslabelshift",
     dr_weightrescale_false_labelshift_true_abstention_results,
     methodnames_yesadapt)]:

  print(settingname)
  abstfrac_to_methodname_to_perfs =\
    results_dict["abstfrac_to_methodname_to_perfs"]
  methodprefix_to_baseperfs = results_dict["methodprefix_to_baseperfs"]
  
  abstfrac_to_bestmethods = {}  
  for abstfrac in abstfracs:    
    nsamples = len(list(abstfrac_to_methodname_to_perfs[abstfrac].values())[0])
    
    ustats_mat = get_ustats_mat(
        method_to_perfs=abstfrac_to_methodname_to_perfs[abstfrac],
        method_names=methodnames,
        max_ustat=nsamples*(nsamples+1)/2 )
    tied_top_methods, tied_worst_methods =(
        get_tied_top_and_worst_methods(
            ustats_mat,
            methodnames,
            #0.05 threshold for one-sided test from normal approx when N=30,
            #from https://www.oreilly.com/library/view/nonparametric-statistics-a/9781118840429/bapp02.xhtml
            threshold=151
        ))
    tied_top_methods = [methodnames[x] for x in tied_top_methods]
    print(abstfrac, tied_top_methods)
    abstfrac_to_bestmethods[abstfrac] = set(tied_top_methods)
    
  thetable = ("""
\\begin{table*}[!h]
\\tiny
\\begin{center}
\\begin{tabular}{ | c c | c c c c c c | }
\\hline
\\multicolumn{2}{| c |}{ } & \\multicolumn{6}{ c |}{"""
+"Diabetic Retinopathy Cohen's Weighted Kappa "
+("(with label shift)" if ("yeslabelshift" in settingname) else "(no label shift)")+"""}\\\\
\\multicolumn{1}{| c}{Method} & \\multicolumn{1}{c |}{"""
+("Calib" if ("nolabelshift" in settingname) else "Adapted")
+"""?} & Base & @30\\% Abst. & @25\\% Abst. & @20\\% Abst. & @15\\% Abst. & @10\\% Abst. \\\\ \\hline\n""")

  for methodname in methodnames:
    nicename = abstname_to_nicename[methodname.split(":")[1]]
    thetable += nicename
    thetable += " & "+("Y" if ("yes" in methodname.split(":")[0]) else 'N')
    baseperfs = methodprefix_to_baseperfs[methodname.split(":")[0]]
    thetable += (" & "+str(np.round(np.mean(baseperfs),4))
                     +"$\\pm$"+str(np.round(stdderr(baseperfs),4)))
    for abstfrac in abstfracs:
      methodperfs = abstfrac_to_methodname_to_perfs[abstfrac][methodname]
      isbest = (methodname in abstfrac_to_bestmethods[abstfrac])
      thetable += " & "
      if (isbest):
        thetable += "\\textbf{"
      thetable += (str(np.round(np.mean(methodperfs),4))
                       +"$\pm$"+str(np.round(stdderr(methodperfs),4)))
      if (isbest):
        thetable += "}"
    thetable += "\\\\ \\hline\n"
    if (methodname=="yescalib:jsdiv" or methodname=="noadapt:jsdiv"):
      thetable += "\\hline\n"
  thetable += """\\end{tabular}
\\end{center}
\\caption{\\textbf{}. }
\\label{tab:dr_"""+settingname+"""}
\\end{table*}
"""
  print(thetable)
   

yesweightrescale_nolabelshift
0.3 ['yescalib:estmarginalweightedkappa']
0.25 ['yescalib:estmarginalweightedkappa']
0.2 ['yescalib:estmarginalweightedkappa']
0.15 ['yescalib:estmarginalweightedkappa']
0.1 ['yescalib:estmarginalweightedkappa']

\begin{table*}[!h]
\tiny
\begin{center}
\begin{tabular}{ | c c | c c c c c c | }
\hline
\multicolumn{2}{| c |}{ } & \multicolumn{6}{ c |}{Diabetic Retinopathy Cohen's Weighted Kappa (no label shift)}\\
\multicolumn{1}{| c}{Method} & \multicolumn{1}{c |}{Calib?} & Base & @30\% Abst. & @25\% Abst. & @20\% Abst. & @15\% Abst. & @10\% Abst. \\ \hline
Est $\Delta$Metric (Ours) & Y & 0.8104$\pm$0.0019 & \textbf{0.8918$\pm$0.0013} & \textbf{0.8875$\pm$0.0014} & \textbf{0.88$\pm$0.0016} & \textbf{0.8714$\pm$0.0016} & \textbf{0.8597$\pm$0.0017}\\ \hline
JS Div. (Ours) & Y & 0.8104$\pm$0.0019 & 0.8529$\pm$0.0015 & 0.8497$\pm$0.0015 & 0.8463$\pm$0.0015 & 0.8419$\pm$0.0015 & 0.8357$\pm$0.0015\\ \hline
\hline
Entropy & Y & 0.8104$\pm$0.0019 & 0.7667$\pm$0.0064

In [10]:
dr_weightrescale_false_labelshift_true_abstention_results['methodprefix_to_baseperfs'].keys()

dict_keys(['noadapt', 'yesadapt'])

In [0]:
#note: in some cases, the uncalibrated version of
# estmarginalweightedkappa does better than the calibrated version, but
# this is likely an artifact of having access to only one pretrained model
# (unfortunately the other winners didn't make their models public, to our knowledge)